In [2]:
# my_label_path="/home/aikedaer/mydata/data/lingodt/val.csv"
# my_image_path="/home/aikedaer/mydata/data/lingodt"

my_label_path="/home/aikedaer/mydata/data/comp/question1.csv"
my_image_path="/home/aikedaer/mydata/data/comp"
qton = "question1"

# my_label_path="/home/aikedaer/mydata/data/comp/question2.csv"
# my_image_path="/home/aikedaer/mydata/data/comp/images/phase2"
# qton = "question2" 

In [3]:
!pip show transformers

Name: transformers
Version: 4.39.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /mnt/data1/aikedaer/anaconda3/envs/imp/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: transformers-stream-generator


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
# torch.set_default_device("cuda")
default_device = torch.device("cuda:4")
torch.cuda.set_device(default_device)
torch.__version__


'2.3.0+cu121'

In [5]:
# !pip install transformers_stream_generator -i https://pypi.tuna.tsinghua.edu.cn/simple

In [6]:
# !pip install tiktoken matplotlib -i https://pypi.tuna.tsinghua.edu.cn/simple

In [7]:
# !pip install torchvision

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
torch.manual_seed(1234)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL", trust_remote_code=True)

# use bf16
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="cuda:6", trust_remote_code=True, bf16=True).eval()
# use fp16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL", device_map="auto", trust_remote_code=True, fp16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL", device_map="cpu", trust_remote_code=True).eval()
# use cuda device
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL", device_map="cuda", trust_remote_code=True).eval()




/mnt/data1/aikedaer/anaconda3/envs/imp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 10/10 [00:18<00:00,  1.86s/it]


In [9]:
import pandas as pd
import os
from tqdm import tqdm 

def get_answer(img_path, question):
    file_pth = os.path.join(my_image_path,img_path)

    query = tokenizer.from_list_format([
        {'image': file_pth},
        {'text': question},
    ])

    response, history = model.chat(tokenizer, query=query, history=None)

    return response

def str2list(input_string):
    # Remove the brackets and split by newline character
    list_elements = input_string.strip("[]").split('\n')

    # Remove any extra spaces and quotes
    cleaned_list = [element.strip(" '") for element in list_elements]
    return cleaned_list

val = pd.read_csv(my_label_path)
val["images"] = val["images"].map(str2list)
predictions = pd.DataFrame(columns=["question_id", "segment_id", "answer"])

for index, row in tqdm(val.iterrows(), total=val.shape[0]):
    question_id = row['question_id']
    segment_id = row['segment_id']
    images = row['images']
    question = row['question']
    answer = row['answer']
    # only use the first frame
    try:
        asw = get_answer(images[0], question)
    except Exception as e:
        asw = f"An exception occured: {e}"
    # import pdb;pdb.set_trace()
    # Create a DataFrame for the current row
    new_row = pd.DataFrame({
        "question_id": [question_id],
        "segment_id": [segment_id],
        # "question": [question],
        "answer": [asw]
    })

    # Append the new row to the predictions DataFrame using pd.concat
    predictions = pd.concat([predictions, new_row], ignore_index=True)
    # break

predictions



100%|██████████| 922/922 [17:17<00:00,  1.13s/it]


,question_id,segment_id,answer
0,xxxxxxxxxxx_0,xxxxxxxxxxxxx_0,"There is no existence of cars, traffic lights,..."
1,xxxxxxxxxxx_1,xxxxxxxxxxxxx_1,"There is no existence of cars, traffic lights,..."
2,xxxxxxxxxxx_2,xxxxxxxxxxxxx_2,"There is no existence of any cars, traffic lig..."
3,xxxxxxxxxxx_3,xxxxxxxxxxxxx_3,"There is a red car, yellow traffic lights, a g..."
4,xxxxxxxxxxx_4,xxxxxxxxxxxxx_4,"There are no cars, traffic lights, road signal..."
...,...,...,...
917,xxxxxxxxxxx_917,xxxxxxxxxxxxx_917,"The cars are black, the traffic lights are yel..."
918,xxxxxxxxxxx_918,xxxxxxxxxxxxx_918,"There is no existence of cars, traffic lights,..."
919,xxxxxxxxxxx_919,xxxxxxxxxxxxx_919,"There is no existence of cars, traffic lights,..."
920,xxxxxxxxxxx_920,xxxxxxxxxxxxx_920,"There is no existence of cars, traffic lights,..."


In [10]:
predictions.to_csv(my_label_path.replace(qton, qton+"_qwen_predict"))